In [1]:
# Uncomment and run these commands if the packages are not installed.
# !pip install beautifulsoup4 requests textblob googletrans==4.0.0-rc1 gTTS nltk pandas

# Import required libraries
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from googletrans import Translator
from gtts import gTTS
import os
import uuid
import pandas as pd

# For displaying audio in Jupyter (if supported)
from IPython.display import Audio, display


In [2]:
def fetch_news(company):
    """
    Fetch news articles related to the given company using Google News RSS feed.
    Returns a list of dictionaries with article title, summary, and link.
    """
    url = f"https://news.google.com/rss/search?q={company}"
    articles = []
    try:
        response = requests.get(url)
        # Parse the XML content
        soup = BeautifulSoup(response.content, "xml")
        items = soup.find_all("item")
        count = 0
        for item in items:
            if count >= 10:
                break
            title = item.title.text if item.title else "No Title"
            summary = item.description.text if item.description else "No Summary"
            link = item.link.text if item.link else ""
            articles.append({
                "title": title,
                "summary": summary,
                "link": link
            })
            count += 1
    except Exception as e:
        print(f"Error fetching news: {e}")
    return articles

# Test the fetch_news function
company = input('Enter Company Name: ')
articles = fetch_news(company)
print(f"Fetched {len(articles)} articles for {company}")
for idx, art in enumerate(articles, 1):
    print(f"{idx}. {art['title']}")


Enter Company Name:  Tesla


Fetched 10 articles for Tesla
1. Minnesota Guv Tim Walz clarifies Tesla stock remarks, takes fresh swipe at Elon Musk - Moneycontrol
2. 'Tesla growth isn't slowing down': Startup founder says Tesla's service centers 'barely keeping up' with d - The Economic Times
3. Tata Group partners with Tesla: A new era for Indian electric vehicle supply chains - The Economic Times
4. Tesla’s entry into India: A Catalyst or a Niche Player? - Deccan Herald
5. 2 Tesla Cars Confirmed For India! - MotorOctane
6. Donald Trump Says Tesla Vandals Could Serve 20 Years In El Salvador Jail - NDTV
7. New day, new problem: Tesla owners ditch EVs in record numbers as Elon Musk’s political ties with Donald T - The Economic Times
8. Kash Patel's FBI backs Musk, sounds alarm over rising attacks on Tesla dealerships - The Times of India
9. Why Tesla recalled nearly all Cybertrucks in US, in the latest of the model’s many woes - The Indian Express
10. Tesla recalls over 46,000 Cybertrucks over safety risk as panel d

In [3]:
def analyze_sentiments(text):
    """
    Analyze the sentiment of the given text.
    Returns a sentiment classification: "Positive", "Negative", or "Neutral".
    """
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

# Test the sentiment analysis function on a sample text
sample_text = "Tesla's latest model has received rave reviews for its innovative features and performance."
print("Sentiment:", analyze_sentiments(sample_text))


Sentiment: Positive


In [4]:
def generate_comparative_analysis(articles):
    """
    Generate a comparative analysis from the list of articles.
    Returns a dictionary with sentiment distribution, a sample coverage difference, and topic overlap.
    """
    sentiment_counts = {"Positive": 0, "Negative": 0, "Neutral": 0}
    for art in articles:
        sentiment = art.get("sentiment", "Neutral")
        sentiment_counts[sentiment] = sentiment_counts.get(sentiment, 0) + 1

        # Dummy topic extraction: get first 3 unique words from the summary
        words = art.get("summary", "").split()
        art["topics"] = list(dict.fromkeys(words))[:3]

    # Create a sample coverage difference between first two articles (if available)
    coverage_differences = []
    if len(articles) >= 2:
        coverage_differences.append({
            "Comparison": f"Article 1 ('{articles[0]['title']}') focuses on one aspect, while Article 2 ('{articles[1]['title']}') covers another.",
            "Impact": "This shows mixed market perceptions."
        })

    # Calculate topic overlaps for the first two articles (if available)
    topics_article1 = set(articles[0].get("topics", [])) if articles else set()
    topics_article2 = set(articles[1].get("topics", [])) if len(articles) > 1 else set()
    common_topics = list(topics_article1.intersection(topics_article2))
    unique_topics_article1 = list(topics_article1 - topics_article2)
    unique_topics_article2 = list(topics_article2 - topics_article1)

    topic_overlap = {
        "Common Topics": common_topics,
        "Unique Topics in Article 1": unique_topics_article1,
        "Unique Topics in Article 2": unique_topics_article2
    }
    return {
        "Sentiment Distribution": sentiment_counts,
        "Coverage Differences": coverage_differences,
        "Topic Overlap": topic_overlap
    }

# First, add sentiment analysis to each article
for art in articles:
    art["sentiment"] = analyze_sentiments(art["summary"])

comparative_data = generate_comparative_analysis(articles)
print("Comparative Analysis:")
print(comparative_data)


Comparative Analysis:
{'Sentiment Distribution': {'Positive': 4, 'Negative': 0, 'Neutral': 6}, 'Coverage Differences': [{'Comparison': "Article 1 ('Minnesota Guv Tim Walz clarifies Tesla stock remarks, takes fresh swipe at Elon Musk - Moneycontrol') focuses on one aspect, while Article 2 (''Tesla growth isn't slowing down': Startup founder says Tesla's service centers 'barely keeping up' with d - The Economic Times') covers another.", 'Impact': 'This shows mixed market perceptions.'}], 'Topic Overlap': {'Common Topics': ['<a'], 'Unique Topics in Article 1': ['target="_blank">Minnesota', 'href="https://news.google.com/rss/articles/CBMi1wFBVV95cUxQR3BlWjhyYVczVTRrNzQ4OGtoSGw4QmRuVDgxTVZabzNweWNCTHFrSFhqa3BPVWZ1UG5FRGJ2ZHBVYjlYdXpBZVVoTGxNWjdDYXNmMG9nOXE3UG01c2gxV2pOT1NqTXNtUU9oM0RHaUNkQm83MHFPLXBaTWFvNG5IWV9YZWFDS3dWSURjZWQtODZDTndvODQtS1NhNWRfRnlNY1p6WjQ0OUJLLV9vVEc4V1ZNZTU1OTBtQUtpRWhaYm5UbVBUWWNvSnhBWnNrdXpudXpOV0diSdIB3AFBVV95cUxQVHN0X0Yxck9XZW9BSGFzWklHVlowc2VKTEpSVVJucFR3Vi1sdjhLND

In [5]:
def generate_hindi_tts(text):
    """
    Translate the input text to Hindi and generate a TTS audio file.
    Returns the file path of the generated audio.
    """
    translator = Translator()
    try:
        translation = translator.translate(text, dest='te')
        hindi_text = translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        hindi_text = text  # Fallback if translation fails

    try:
        tts = gTTS(hindi_text, lang="te")
        # Create a unique filename and ensure the 'static' directory exists
        if not os.path.exists("static"):
            os.makedirs("static")
        filename = f"static/audio_{uuid.uuid4().hex}.mp3"
        tts.save(filename)
        return filename
    except Exception as e:
        print(f"TTS generation error: {e}")
        return ""

# Generate a summary text for TTS
summary_text = " ".join([f"{art['title']} shows a {art['sentiment']} sentiment." for art in articles])
audio_file = generate_hindi_tts(summary_text)
print("Generated TTS audio file:", audio_file)

# If running in a Jupyter Notebook, display the audio file if it exists.
if os.path.exists(audio_file):
    display(Audio(audio_file, autoplay=False))


Generated TTS audio file: static/audio_7995e253e35a493f8d48d0644d05729f.mp3


In [6]:
# Integrate all functions into a structured report
def generate_news_report(company):
    articles = fetch_news(company)
    if not articles:
        return {"error": "No articles found."}
    
    # Apply sentiment analysis on each article
    for art in articles:
        art["sentiment"] = analyze_sentiments(art["summary"])
    
    # Generate comparative analysis
    comparative_data = generate_comparative_analysis(articles)
    
    # Generate summary text for TTS and produce audio
    summary_text = " ".join([f"{art['title']} shows a {art['sentiment']} sentiment." for art in articles])
    tts_file = generate_hindi_tts(summary_text)
    
    report = {
        "Company": company,
        "Articles": articles,
        "Comparative Sentiment Score": comparative_data,
        "Final Sentiment Analysis": f"{company} news coverage indicates mixed sentiments based on analyzed articles.",
        "Audio": tts_file
    }
    return report

# Generate report for a sample company (e.g., Tesla)
company = "Tesla"
news_report = generate_news_report(company)
print("News Report for", company)
print(pd.json_normalize(news_report))

# Optionally, display the audio file if it exists
if news_report.get("Audio") and os.path.exists(news_report["Audio"]):
    display(Audio(news_report["Audio"], autoplay=False))


News Report for Tesla
  Company                                           Articles  \
0   Tesla  [{'title': 'Minnesota Guv Tim Walz clarifies T...   

                            Final Sentiment Analysis  \
0  Tesla news coverage indicates mixed sentiments...   

                                               Audio  \
0  static/audio_fc430d95403040689f48c2a8b363e3ef.mp3   

   Comparative Sentiment Score.Sentiment Distribution.Positive  \
0                                                  4             

   Comparative Sentiment Score.Sentiment Distribution.Negative  \
0                                                  0             

   Comparative Sentiment Score.Sentiment Distribution.Neutral  \
0                                                  6            

    Comparative Sentiment Score.Coverage Differences  \
0  [{'Comparison': 'Article 1 ('Minnesota Guv Tim...   

  Comparative Sentiment Score.Topic Overlap.Common Topics  \
0                                               [<a]

In [7]:
# Convert articles into a DataFrame for better visualization
df_articles = pd.DataFrame(news_report["Articles"])
print("Articles DataFrame:")
display(df_articles)


Articles DataFrame:


,title,summary,link,sentiment,topics
0,Minnesota Guv Tim Walz clarifies Tesla stock r...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMi1wFBV...,Positive,"[<a, href=""https://news.google.com/rss/article..."
1,'Tesla growth isn't slowing down': Startup fou...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMimwJBV...,Neutral,"[<a, href=""https://news.google.com/rss/article..."
2,Tata Group partners with Tesla: A new era for ...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMi7AFBV...,Positive,"[<a, href=""https://news.google.com/rss/article..."
3,Tesla’s entry into India: A Catalyst or a Nich...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMimwFBV...,Neutral,"[<a, href=""https://news.google.com/rss/article..."
4,2 Tesla Cars Confirmed For India! - MotorOctane,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMid0FVX...,Positive,"[<a, href=""https://news.google.com/rss/article..."
5,Donald Trump Says Tesla Vandals Could Serve 20...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMisgFBV...,Neutral,"[<a, href=""https://news.google.com/rss/article..."
6,"New day, new problem: Tesla owners ditch EVs i...","<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMiqAJBV...,Neutral,"[<a, href=""https://news.google.com/rss/article..."
7,"Kash Patel's FBI backs Musk, sounds alarm over...","<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMi5gFBV...,Neutral,"[<a, href=""https://news.google.com/rss/article..."
8,Why Tesla recalled nearly all Cybertrucks in U...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMikwFBV...,Positive,"[<a, href=""https://news.google.com/rss/article..."
9,"Tesla recalls over 46,000 Cybertrucks over saf...","<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMi8gFBV...,Neutral,"[<a, href=""https://news.google.com/rss/article..."
